# Test case multi-ecehlon inventory control program
Welcome to this test case of the multi-echelon model implemented within the scope of the Master Thesis called "Comparing single- and multi-echelon methods for inventory control of spare parts at Volvo".

Authors of this project is Jakob Bengtsson and Alexander Larsson, supervisors were Christian Beckers and Johan Lidvall.

This test case displays how to use to program in order to generate optimal reorder-points for an (R,Q) - policy for the example item present in the South African network.

A walkthrough of this test case can give you a brief understanding of the setup of the program and how pieces are connected. However, to really understand what's going on we refer to the theory section (Section 4) of the associated thesis report.


In [39]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *



## Initating inputs
A test case is submitted in the directory "Test_Case". If you have other cases to try out, please enter them here. Remark that even if demand size data is not used (which is the case if demand distribution is Normal, Poisson or NBD) this input is still required by the program. A workaround is to put a placeholder file with demand size of 1 haing probability 100 % as this will not have any effect on the results.

The capital cost used is in percentage per day. The value of 0.15 was given by the supervisors as the holding cost rate in the South African network.

In [40]:
indata_path = "test_case_data/test_case_indata.csv"
indata_demand_size_dist_path = "Test_Case/test_case_indata_demand_sizes.csv"
outdata_path = "test_case_data/test_case_outdata.csv"
capital_cost = 0.15/365

Inputs are handled and converted to arrays in the cell below. Note that there are several columns required to exist in the indata file.

In [41]:

## Handling Inputs
# --------------------------------------------------------------------------       
# Ensuring indata path is not the same as outdata (outdata will erase current file.)
if indata_path == outdata_path:
    raise ValueError('Indata path and outdata path should not be the same.')

indataDF = pd.read_csv(indata_path)

outdataDF = indataDF.copy()

# Ensure correct columns are present:
required_columns = {"Installation id", "Type", "Name", "Transport time", "Q", 
    "Unit cost", "Target item fill rate", "Demand distribution",	"Demand mean per time unit", "Demand stdev per time unit"}
    
if not required_columns.issubset(set(indataDF.columns.to_list())):
    raise ValueError("Indata doesn't contain all required fields, see documentation.")

# Retrieving the data about dealers.
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy().astype("int32")
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean per time unit").to_numpy().astype("float64")
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand distribution").to_numpy()
h_dealer_arr = capital_cost * indataDF.get(indataDF["Type"] == "Dealer").get("Unit cost").to_numpy().astype("float64")
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy().astype("float64")
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy().astype("float64")

# If demand distribution is poisson, retrieve 
sigma_dealer_list = []
for id in indataDF.get(indataDF["Type"] == "Dealer").get("Installation id"):
    if str(indataDF.get(indataDF["Installation id"] == id).get("Demand distribution")) == "Poisson":
        sigma_dealer_list.append(math.sqrt(
            float(indataDF.get(outdataDF["Installation id"]== id).get("Demand mean per time unit"))))
    else:
        sigma_dealer_list.append(float(indataDF.get(indataDF["Installation id"] == id).get("Demand stdev per time unit")))        
sigma_dealer_arr = np.array(sigma_dealer_list)

# Input compounding distribution arrays here!
# Supposed to have one row per dealer.
if indata_demand_size_dist_path[-3:] == "csv":
    compounding_dist_df = pd.read_csv(indata_demand_size_dist_path)
else:
    compounding_dist_df = pd.read_excel(indata_path,indata_demand_size_dist_path)

compounding_dist_matrix = compounding_dist_df.to_numpy().T[1:] # Each array is a column in excel, transposing and removing first row holding item amounts.
compounding_dist_matrix = np.nan_to_num(compounding_dist_matrix,copy = True)


FileNotFoundError: [Errno 2] No such file or directory: 'Test_Case/test_case_indata_demand_sizes.csv'

In [ ]:
compounding_dist_matrix

array([[0.92523364, 0.01869159, 0.01869159, 0.00934579, 0.00934579,
        0.00934579, 0.        , 0.0046729 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.0046729 ],
       [0.98333333, 0.01666667, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.99019608, 0.        , 0.        , 0.        , 0.        ,
        0.00980392, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.98039216, 0.00980392, 0.        , 0.00980392, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0

## Regional distribution center demand
The demand at the RDC (or central warehouse in the network) depend on the inventory policies at dealers. 

First, as dealers are only demanding Q units, the warehouse demand during the leadtime can only be integer multiples of the smalles common divisor of the dealers order quantities.


In [ ]:
#Find the smallest common divisor of Q.
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
print(f"Smallest common divisor of order quantities is: {Q_subbatch_size}")

Smallest common divisor of order quantities is: 1


In [ ]:
   
#Read warehouse values.
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))
h_rdc = capital_cost*float(indataDF.get(indataDF["Type"] == "RDC").get("Unit cost"))
Q_0 = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))/Q_subbatch_size) # Observe, Q_0 is in subbatches.

print(f"Leadtime from CDC to RDC is: {L_wh}, holding cost per day at rdc is: {h_rdc} and order quantity is {Q_0}")

Leadtime from CDC to RDC is: 58.0, holding cost per day at rdc is: 0.12380547945205478 and order quantity is 29


With inputs present and Q_subbatch computed, it is time to find the warehouse demand array. This is done in the function "warehouse_subbatch_demand_probability_array" which you can find in directory "warheouse_modeling". The workings of this is thouroughly described in section 4.10 of the report and detailed references can be found in every part of the function in the "warehouse modeling" directory.

In [ ]:
  
# Central warehouse demand
# --------------------------------------------------------------------------
# Computing subbatch demand probability array, distribution, lead time demand mean, and 
# lead time demand variance at central warehouse.
# Observe that these values are returned in "subbatches".
rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(
Q_dealer_arr, mu_dealer_arr, sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size, 
compounding_dist_matrix)
outdataDF.loc[outdataDF["Type"] == "RDC","Demand distribution"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean per time unit"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev per time unit"] = math.sqrt(sigma2_L * Q_subbatch_size/L_wh)
 
print(f"""The demand probability is interpreted like this:

Probability of lead time demand = 0*Q_subbatch is {rdc_f_u_probability_array[0]}
Probability of lead time demand = 1*Q_subbatch is {rdc_f_u_probability_array[1]}
Probability of lead time demand = 2*Q_subbatch is {rdc_f_u_probability_array[2]}
...""")

2022-06-15 11:10:38,687 - default_log - DEBUG - new_function
2022-06-15 11:10:38,691 - sparse_log - DEBUG - warehouse_subbatch_demand_probability_array
2022-06-15 11:10:38,695 - default_log - DEBUG - warehouse_demand_mean_approximation
2022-06-15 11:10:38,697 - default_log - DEBUG - warehouse_demand_mean_approximation ended.
2022-06-15 11:10:38,699 - default_log - DEBUG - warehouse_demand_variance_approximation
2022-06-15 11:10:38,701 - default_log - DEBUG - warehouse_demand_variance_term
2022-06-15 11:10:38,703 - default_log - DEBUG - pmf_func_warehouse_subbatch_demand
2022-06-15 11:10:45,604 - default_log - DEBUG - delta_func_Empiric_Compound_Poisson_demand
2022-06-15 11:10:45,607 - default_log - DEBUG - delta_func_Empiric_Compound_Poisson_demand ended.
2022-06-15 11:10:45,618 - default_log - DEBUG - delta_func_Empiric_Compound_Poisson_demand
2022-06-15 11:10:45,631 - default_log - DEBUG - delta_func_Empiric_Compound_Poisson_demand ended.
2022-06-15 11:10:45,633 - default_log - DEBUG

The demand probability is interpreted like this:

Probability of lead time demand = 0*Q_subbatch is 2.177405732273665e-19
Probability of lead time demand = 1*Q_subbatch is 6.015553201084338e-18
Probability of lead time demand = 2*Q_subbatch is 8.495198857399001e-17
...


## Induced backorder cost
When lead time demand at the warehouse is found. The induced backorder cost is required in order to find optimal reorder points R.

In [ ]:
   
# Calculating induced backorder cost.
# --------------------------------------------------------------------------
# Computing shortage costs.
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
    
# Computing induced backorder cost for each retailer.
beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,
mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))
beta_arr = np.array(beta_list)

# Computing weighted backorder cost at central warehouse.
mu_wh = mu_L/L_wh * Q_subbatch_size
beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
    
outdataDF.loc[outdataDF["Type"] == "RDC", "Holding cost"] = h_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Holding cost"] = h_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Estimated shortage cost"] = p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "RDC", "Beta"] = beta_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Beta"] = beta_arr

print(f"The induced backorder cost at the RDC is: {beta_rdc}")

2022-06-15 11:11:51,823 - default_log - DEBUG - induced_backorder_cost_opt
2022-06-15 11:11:51,828 - default_log - DEBUG - norm_sigma
2022-06-15 11:11:51,833 - default_log - DEBUG - norm_sigma ended.
2022-06-15 11:11:51,835 - default_log - DEBUG - induced_backorder_cost_opt ended.
2022-06-15 11:11:51,836 - default_log - DEBUG - induced_backorder_cost_opt
2022-06-15 11:11:51,838 - default_log - DEBUG - norm_sigma
2022-06-15 11:11:51,840 - default_log - DEBUG - norm_sigma ended.
2022-06-15 11:11:51,842 - default_log - DEBUG - induced_backorder_cost_opt ended.
2022-06-15 11:11:51,848 - default_log - DEBUG - induced_backorder_cost_opt
2022-06-15 11:11:51,850 - default_log - DEBUG - norm_sigma
2022-06-15 11:11:51,854 - default_log - DEBUG - norm_sigma ended.
2022-06-15 11:11:51,855 - default_log - DEBUG - induced_backorder_cost_opt ended.
2022-06-15 11:11:51,859 - default_log - DEBUG - induced_backorder_cost_opt
2022-06-15 11:11:51,861 - default_log - DEBUG - norm_sigma
2022-06-15 11:11:51,

The induced backorder cost at the RDC is: 0.28001674915125674


In order to provide a better understanding of the induced backorder cost we here translate is to a fill rate value.

In [ ]:

# Calculating target fill rate corresponding to beta estimate
target_fill_rate_warehouse = beta_rdc/(beta_rdc + h_rdc)
outdataDF.loc[outdataDF["Type"]== "RDC", "Target item fill rate"] = target_fill_rate_warehouse

print(f"The induced backorder cost of {round(beta_rdc,4)} gives a similar reorder point as setting the target fill rate to approximately {100*round(target_fill_rate_warehouse,2)} %")

The induced backorder cost of 0.28 gives a similar reorder point as setting the target fill rate to approximately 69.0 %


## Optimizing reorder point at RDC/central warehouse

The optimal reorder point is computed by balancing costs of holding inventory and costs of backorders. The costs of holding inventory increase with more inventory. Backorder costs increase with less inventory. 

Here, the induced backorder costs is used to put a value on the backorders.

In [ ]:

# Optimizing reorder point at central warehouse
# --------------------------------------------------------------------------
# Computing optimal reorder points as well as corresponding expected stock on hand 
# and backorders.
R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)

outdataDF.loc[outdataDF["Type"] == "RDC", "R optimal"] = R_0*Q_subbatch_size
stock_on_hand_wh = positive_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdataDF.loc[outdataDF["Type"] == "RDC","Expected stock on hand"] = stock_on_hand_wh
    
backorders_wh = negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdataDF.loc[outdataDF["Type"] == "RDC","Expected backorders"] = backorders_wh

print(f"Optimal (R,Q)-policy at RDC is R = {R_0} and Q = {Q_0} ")

2022-06-15 11:18:27,495 - default_log - DEBUG - new_function
2022-06-15 11:18:27,506 - sparse_log - DEBUG - warehouse_optimization
2022-06-15 11:18:27,512 - default_log - DEBUG - total_cost
2022-06-15 11:18:27,514 - default_log - DEBUG - positive_inventory
2022-06-15 11:18:27,526 - default_log - DEBUG - positive_inventory ended.
2022-06-15 11:18:27,527 - default_log - DEBUG - negative_inventory
2022-06-15 11:18:27,531 - default_log - DEBUG - negative_inventory ended.
2022-06-15 11:18:27,533 - default_log - DEBUG - total_cost ended.
2022-06-15 11:18:27,535 - default_log - DEBUG - total_cost
2022-06-15 11:18:27,536 - default_log - DEBUG - positive_inventory
2022-06-15 11:18:27,538 - default_log - DEBUG - positive_inventory ended.
2022-06-15 11:18:27,539 - default_log - DEBUG - negative_inventory
2022-06-15 11:18:27,544 - default_log - DEBUG - negative_inventory ended.
2022-06-15 11:18:27,547 - default_log - DEBUG - total_cost ended.
2022-06-15 11:18:27,549 - default_log - DEBUG - total_c

Optimal (R,Q)-policy at RDC is R = 65 and Q = 29 


## Computing delay- and lead time
With the optimal policy in place at the RDC, it is time to compute the actual lead time between RDC and dealers depending on the transportation time between installaitons and the delay due to stockouts at the RDC.


In [ ]:

# Computing expected delay and lead time
# --------------------------------------------------------------------------
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Expected delay"] = W
lead_time_dealer_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = lead_time_dealer_arr

# Entering dealer lead time demand and standard deviation.
mu_L_dealer_array = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time").to_numpy()*outdataDF.get(
        outdataDF["Type"] == "Dealer").get("Demand mean per time unit").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = mu_L_dealer_array
    
sqrt_dealer_lead_time_arr = np.sqrt(lead_time_dealer_arr)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] = sqrt_dealer_lead_time_arr*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev per time unit").to_numpy()
    
# Entering central warehouse lead time as transport time for completeness.
outdataDF.loc[outdataDF["Type"] == "RDC", "Lead time"] = indataDF.get(indataDF["Type"] == "RDC").get("Transport time")

# Computing MTBA (mean time between arrivals) for use in simulation.
# reference: Axsäter, 2006, Inventory control, eq. 5.4
MTBA_arr = np.zeros_like(mu_L_dealer_array)
for i,mu in enumerate(mu_L_dealer_array):
    compounding_dist_arr = compounding_dist_matrix[i]
    j_arr = np.arange(start=1,stop=len(compounding_dist_arr)+1)
    lam = mu/j_arr.dot(compounding_dist_arr)
    MTBA_arr[i] = 1/lam*lead_time_dealer_arr[i]
outdataDF.loc[outdataDF["Type"]== "Dealer", "MTBA"] = MTBA_arr


2022-06-15 11:20:16,752 - default_log - DEBUG - negative_inventory
2022-06-15 11:20:16,774 - default_log - DEBUG - negative_inventory ended.


## Optimizing reorder points at the dealer
Finally, it is time to find optimal (R,Q)-policies at the dealer by minimizing the holding costs under target service level constraints. The actual optimization is done in "dealer_R_optimization()"


In [ ]:

# Optimizing reorder points at dealer.
# --------------------------------------------------------------------------
# Computing optimal reorder point, expected realised fill rate, and expected 
# stock on hand level.
opt_dealer_list = []
for Q,L_est,fill_rate_target,demand_type,mu,sigma,compounding_dist_arr in zip(Q_dealer_arr,
    lead_time_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr, compounding_dist_matrix):
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,
        mu,demand_variance = math.pow(sigma,2),compounding_dist_arr=compounding_dist_arr))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[1])
    exp_stock_on_hand_list.append(tup[2])
R_opt_dealer_arr = np.array(R_opt_dealer_list)
fill_rate_dealer_arr = np.array(fill_rate_dealer_list)
exp_stock_on_hand_arr = np.array(exp_stock_on_hand_list)
    

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_arr
    

# Adding expected backorders at retailers.
exp_backorders_dealer_arr = [ 
    expected_backorders_discrete(R,Q,lt_mu,exp_stock_on_hand) for 
    R,Q,lt_mu,exp_stock_on_hand in zip(R_opt_dealer_list,Q_dealer_arr,
    outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time demand mean").to_numpy(),
    exp_stock_on_hand_list) ]
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected backorders"] = exp_backorders_dealer_arr


## Last touches and output

In [ ]:

# Computing cost expressions
# --------------------------------------------------------------------------
total_holding_cost_dealers_arr = h_dealer_arr*exp_stock_on_hand_arr
total_backorder_cost_dealers_arr = exp_backorders_dealer_arr*p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected holding costs per time unit"] = total_holding_cost_dealers_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected shortage costs per time unit"] = total_backorder_cost_dealers_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Total expected costs"] = total_holding_cost_dealers_arr + total_backorder_cost_dealers_arr
    
outdataDF.loc[outdataDF["Type"] == "RDC", "Expected holding costs per time unit"] = h_rdc*stock_on_hand_wh
outdataDF.loc[outdataDF["Type"] == "RDC", "Total expected costs"] = h_rdc*stock_on_hand_wh


In [43]:
# Printing results to CSV.
# --------------------------------------------------------------------------

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

if outdata_path is not None:
    outdataDF.to_csv(outdata_path, index = False)

outdataDF = pd.read_csv(outdata_path, index_col = False)
outdataDF


,Unnamed: 0.1,Unnamed: 0,Installation id,Type,Name,Transport time,Q,Unit cost,Target item fill rate,Demand distribution,Demand mean per time unit,Demand stdev per time unit,Demand type,Stocked,Inventory policy
0,0,0,Bartlett,Dealer,Bartlett,5,9.0,432.90,0.970,Empiric_Compound_Poisson,0.380267,0.641949,Fast,Yes,BABZA_Bartlett_IP
1,1,1,Bloemfontein,Dealer,Bloemfontein,5,3.0,432.90,0.980,Empiric_Compound_Poisson,0.082613,0.264094,Fast,Yes,BABZA_Bloemfontein_IP
2,2,2,Capetown,Dealer,Capetown,5,4.0,432.90,0.980,Empiric_Compound_Poisson,0.132640,0.578724,Fast,Yes,BABZA_Capetown_IP
3,3,3,Durban,Dealer,Durban,5,4.0,432.90,0.980,Empiric_Compound_Poisson,0.112693,0.463830,Fast,Yes,BABZA_Durban_IP
4,4,4,George,Dealer,George,10,2.0,432.90,0.985,Empiric_Compound_Poisson,0.023467,0.153782,Fast,Yes,BABZA_George_IP
5,5,5,Kimberley,Dealer,Kimberley,10,1.0,432.90,0.000,Empiric_Compound_Poisson,0.009090,0.085280,Slow,No,BABZA_Kimberley_IP
6,6,6,Middelburg,Dealer,Middelburg,10,6.0,432.90,0.980,Empiric_Compound_Poisson,0.143733,0.649270,Erratic,Yes,BABZA_Middelburg_IP
7,7,7,Nelspruit,Dealer,Nelspruit,10,3.0,432.90,0.980,Empiric_Compound_Poisson,0.041333,0.192798,Fast,Yes,BABZA_Nelspruit_IP
8,8,8,Port Elizabeth,Dealer,Port Elizabeth,10,3.0,432.90,0.980,Empiric_Compound_Poisson,0.060640,0.317588,Fast,Yes,BABZA_Port Elizabeth_IP
9,9,9,Richards Bay,Dealer,Richards Bay,10,3.0,432.90,0.980,Empiric_Compound_Poisson,0.048107,0.106715,Fast,Yes,BABZA_Richards Bay_IP
